# Load line values

This is a demo that queries a geojson line file and evaluates forecasted near-term fire risk based on the [Wildfire Forecast](https://data.spatiafi.com/dataset/36) dataset.  We start by loading our App Credentials, and then using the [geojson statistics](https://docs.spatiafi.com/api/#operation/geojson_statistics_api_statistics_post) endpoint to fetch fire risk values near the line of interest, including a buffer of 50km.

---

Install `spatiafi` (if not already installed):

In [ ]:
!sudo apt -y install libgeos-dev

In [ ]:
%pip install --upgrade spatiafi cartopy xarray geopandas

In [ ]:
import json

import geopandas as gpd
import matplotlib.pyplot as plt
import spatiafi
from cartopy import crs as ccrs
from cartopy.feature import NaturalEarthFeature
from google.cloud import storage
from shapely.geometry import MultiLineString

In [ ]:
session = spatiafi.get_session()
session

## Loading geojson line

In [ ]:
storage_client = storage.Client()

bucket_name = "ce-datasets"
folder_name = "ce-cn/cn-rail-assets-vector/public/cn_rail_from_ntad"
file_name = "cn_rail_network_from_NTAD.geojson"

bucket = storage_client.bucket(bucket_name)
folder_path = f"{folder_name}/" if folder_name else ""
file_path = f"{folder_path}{file_name}"

blob = bucket.blob(file_path)

if blob.exists():
    blob_content = blob.download_as_text()
    geojson_objects = blob_content.splitlines()

    desired_object_id = 557
    desired_geojson_object = None

    for obj in geojson_objects:
        data = json.loads(obj)
        if data["properties"]["OBJECTID"] == desired_object_id:
            desired_geojson_object = data
            break
    else:
        print(f"GeoJSON object with OBJECTID = {desired_object_id} not found")
else:
    print("GeoJSON file not found in GCS")

In [ ]:
# Extract the geometries from the GeoJSON dictionary
geometries = desired_geojson_object["geometry"]

# Convert the geometries to Shapely objects
shapely_geometries = MultiLineString(geometries["coordinates"])

# Create a GeoDataFrame from the Shapely geometries
gdf = gpd.GeoDataFrame(geometry=[shapely_geometries])

We show here an example of loading in a section of CN rail line. Once we have loaded in the geojson line, we add a buffer of 50km in order to see if there are locations near the rail line at risk of fire.

In [ ]:
# Assuming `desired_geojson_object` contains the desired GeoJSON object


def generate_buffer_meter(data, radiu, geometry="geometry", crs="epsg:4326"):
    data = gpd.GeoDataFrame(data, geometry=geometry, crs=crs)
    data = data.to_crs("+proj=aeqd +units=m  +x_0=0 +y_0=0")
    data[geometry] = data[geometry].buffer(radiu)
    data = data.to_crs(crs)
    return data


buffered = generate_buffer_meter(gdf, 50000)

lons = buffered["geometry"][0].exterior.coords.xy[0]
lats = buffered["geometry"][0].exterior.coords.xy[1]
lon_lat_pairs = list(zip(lons, lats))

# Create a matplotlib figure and axis
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={"projection": ccrs.PlateCarree()})

# Add coastlines from Natural Earth dataset
coastlines = NaturalEarthFeature(
    category="physical", scale="10m", facecolor="none", name="coastline"
)
ax.add_feature(coastlines, edgecolor="gray")

# Set the extent of the map to focus on US and Canada
extent = [-125, -120, 45, 50]  # [min_lon, max_lon, min_lat, max_lat]
extent = [-100, -60, 30, 60]
ax.set_extent(extent, crs=ccrs.PlateCarree())

# Plot the GeoDataFrame on the axis
buffered.plot(ax=ax, color="red")

# Show the plot
plt.show()

In [ ]:
url = "https://api.spatiafi.com/api/statistics"

params = {
    "item_id": "ce-cfsv2-era5-fwi-forecast-day01",
    "bidx": "1",
}

payload = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [lon_lat_pairs],
        "type": "Polygon",
    },
}

# `POST` the request using our `session` object, which will automatically handle authentication.
not_done = True
while not_done:
    response = session.post(url, json=payload, params=params)

    if response.status_code != int(500):
        not_done = False
        data = response.json()

print(data["properties"])

We can see that some locations near the line are at significant risk of fire (with a max value of 0.92. This can help inform where fire mitigation should take place.